In [10]:
import re
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
#from convertTxtToDFFns import parseFullDay
import horseInfoFns
from genInfoFns import parseGenInfo

In [280]:
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.112.txt') as file:
    test1 = file.readlines()
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.148.txt') as file:
    test2 = file.readlines()

In [279]:
parseFullDay(test1)

15Dec18 LRC 4 Spectacular Storm (Vergara, Jr., Octavio) 123 L b 3 5 5 3 1 0.80* svd ground, rail rally

 2 6 21/2 11/2

 7Dec18 LRC 1 Six Eighteen (Payeras, Edgar) 120 L bf 1 4 1 1 2 1.90 inside, clear, caught

 5 7 1/2 1 11/2

 7Dec18 LA 3 Charming Weekend (De La Torre, Julio) 124 - - 2 1 2 2 3 10.20 pressed outside,weaknd

 1 4 2 11/2 4

 7Dec18 LA 5 Empress Songbird (Bednar, Vinnie) 123 L b 4 3 3 4 4 4.40 stalked, no response

 1 3 1 2 3/4

 23Nov18 LA 6 Coach House Girl (Garcia, Erick) 123 L b 5 2 4 5 5 16.80 off rail, no factor

 1 3 2

 30Nov18 LA 5 The Tattoo Kid (Payeras, Edgar) 123 L b 5 1 1 1 1 2.80 set pace, held on

 1 1 2 2 Head

 7Dec18 LA 2 Hes Knot Unusual (Orozco, Edgar) 123 L b 2 3 2 2 2 0.50* angled 3w,rally,missed

 2 2 1/2 2 21/4

 7Dec18 LA 4 Lancasterinvision (Franco, Cesar) 124» L f 4 4 6 5 3 29.50 svd ground, mild rally

 2 4 3 1/2

 16Dec18 LA 3 On the Fence (Aragon, Christian) 123 L 3 5 4 4 4 6.20 3wide, no rally

 5 4 2 1 31/2

 7Dec18 LA 6 Fandarel Dancer (

TypeError: cannot concatenate object of type '<class 'int'>'; only Series and DataFrame objs are valid

In [11]:
def parseFullDay(fullChart):
    newRaceInd = [-1] #first index will be 0 after conversion (controls for out of index error later)
    newRaceTest = 'Copyright 2020 Equibase Company LLC. All Rights Reserved.'
    for i in range(len(fullChart)):
        if re.search(newRaceTest, fullChart[i]) is not None: #check for the expression that signifies end of race
            newRaceInd.append(i)

    dayDF = pd.DataFrame(columns=['trackName','month','day','year','raceNum','distance','surface','segment1','segment2','segment3','segment4','segment5','segments'])

    for i in range(len(newRaceInd) - 1):

        dayDF = pd.concat([dayDF, parseRace(fullChart[newRaceInd[i] + 1:newRaceInd[i + 1]])]) #loop over chunks of the file, each chunk representing a full race

    return dayDF

In [4]:
def parseRace(raceChart):

    cnt = 0

    raceDF = 0 #### NEED TO FILL THIS IN

    #loop to find indexes for different parse sections
    for line in raceChart:
        if re.search('Cancelled - ', line) is not None: #check for cancelled race first, if cancelled, return empty DF
            return raceDF
        elif re.search('- Quarter Horse', line) is not None: #also do not need to process quarter horses
            return raceDF
        elif re.search('Last Raced Pgm', line) is not None: #everything before this falls under "general info"
            genInd = cnt + 1
            horseInd = [cnt + 1]
        elif re.search('Fractional Times:|Final Time:', line) is not None: #after general info, need to process info for each horse
            horseInd.append(cnt)
            timesInd = [cnt]
        elif re.search('Run-Up: ', line) is not None: #after horses, timing and runup info
            timesInd.append(cnt + 1)
            betInd = [cnt + 1]
        elif re.search('Past Performance Running Line Preview', line) is not None: #after timing, betting info and additional horse info
            betInd.append(cnt)
            runLineInd = [cnt]
        elif re.search('Trainers: ', line) is not None: #finally, need to do trainers, owners and other ending info
            runLineInd.append(cnt)
            endInfoInd = cnt
        cnt += 1
    """
    print(raceChart[:genInd])
    print('++++++++++++++++++++++++++++++++++++')
    print(raceChart[horseInd[0]:horseInd[1]])
    print('++++++++++++++++++++++++++++++++++++')
    print(raceChart[timesInd[0]:timesInd[1]])
    print('++++++++++++++++++++++++++++++++++++')
    print(raceChart[betInd[0]:betInd[1]])
    print('++++++++++++++++++++++++++++++++++++')
    print(raceChart[runLineInd[0]:runLineInd[1]])
    print('++++++++++++++++++++++++++++++++++++')
    print(raceChart[endInfoInd:])
    """
    
    genItems = parseGenInfo(raceChart[:genInd])
    horseItems = parseHorseInfo(raceChart[horseInd[0]:horseInd[1]])
    """
    timesItems = parseTimingInfo(raceChart[timesInd[0]:timesInd[1]])
    betItems = parseBetInfo(raceChart[betInd[0]:betInd[1]])
    runLineItems = parseRunLineInfo(raceChart[runLineInd[0]:runLineInd[1]])
    endItems = parseEndInfo(raceChart[endInfoInd:])
    """

    return horseItems
    #genDF = pd.DataFrame(horseItems)
    
    #return genDF

    #make df row, append to df, and return

In [260]:
def parseHorseInfo(horseLines):
    numHorses = int(len(horseLines) / 2)

    #set up empty df
    horseDF = pd.DataFrame(columns = ['lastRaceDay','lastRaceMonth','lastRaceYear','lastRaceTrack','lastRaceNum','lastRacePlace','program','horse', 'Jockey','weight','m_e','placePP','placeSeg1','lengthsSeg1','placeSeg2','lengthsSeg2','placeSeg3','lengthsSeg3','placeSeg4','lengthsSeg4','placeSeg5','lengthsSeg5','placeSeg6','lengthSeg6','odds','comments'])

    horseDict = {}

    priorLineBottom = True
    missingTopLineInd = []
    for i in range(len(horseLines)): #loop to add spacing if top line missing (can only happen if horse has never been raced AND was in last place the whole race)
        line = horseLines[i]
        if re.search('^ (\d?\d[A-Z][a-z]{2}\d\d|---)', line) is not None and priorLineBottom: #check to see if the current line is bottom and the last line was also bottom
            missingTopLineInd.append(i) #add index where true
            priorLineBottom = True
        else:
            priorLineBottom = False

    for i in range(len(missingTopLineInd)): #now need to loop over indices and insert empty spacer lines
        horseLines.insert(missingTopLineInd[i], ' ')
        missingTopLineInd = [x + 1 for x in missingTopLineInd] #need to increment remaining indices to account for new element

    #loop over each horse in the lines provided
    for i in [x*2 for x in list(range(numHorses))]:
        activeHorse = horseLines[i:i+2]
        bottomItems = parseHorseBottomLine(activeHorse[1])
        topItems = parseHorseTopLine(activeHorse[0])

        horseDict['lastRaceDay'] = bottomItems[0]
        horseDict['lastRaceMonth'] = bottomItems[1]
        horseDict['lastRaceYear'] = bottomItems[2]
        horseDict['lastRaceTrack'] = bottomItems[3]
        horseDict['lastRaceNum'] = topItems[0]
        horseDict['lastRacePlace'] = topItems[1]
        horseDict['program'] = bottomItems[4]
        horseDict['horse'] = bottomItems[5]
        horseDict['jockey'] = bottomItems[6]
        horseDict['weight'] = bottomItems[7]
        horseDict['m_e'] = bottomItems[8]
        horseDict['placePP'] = bottomItems[9]
        horseDict['placeSeg1'] = bottomItems[10]
        horseDict['lengthsSeg1'] = topItems[2]
        horseDict['placeSeg2'] = bottomItems[11]
        horseDict['lengthsSeg2'] = topItems[3]
        horseDict['placeSeg3'] = bottomItems[12]
        horseDict['lengthsSeg3'] = topItems[4]
        horseDict['placeSeg4'] = bottomItems[13]
        horseDict['lengthsSeg4'] = topItems[5]
        horseDict['placeSeg5'] = bottomItems[14]
        horseDict['lengthsSeg5'] = topItems[6]
        horseDict['placeSeg6'] = bottomItems[15]
        horseDict['lengthsSeg6'] = topItems[7]
        horseDict['odds'] = bottomItems[16]
        horseDict['comments'] = bottomItems[17]

        activeDF = pd.DataFrame(horseDict, index = [0])
        
        horseDF = pd.concat([horseDF, activeDF])

    return horseDF
            

In [221]:
line = ' 15Dec18 LRC 4 Spectacular Storm (Vergara, Jr., Octavio) 123 L b 3 5 5 3 1 0.80* svd ground, rail rally'
#line = ' 9 11 21/2 7 6 1'
#line = ' 11'
#line = ' --- 9 Midnight Roamer (Quinonez, Alonso) 122 L 8 10 10 10 10 10 9 94.00 off bit slow,inside'

In [220]:
topItems = parseHorseTopLine(line)

In [268]:
bottomItems = parseHorseBottomLine(line)

horseDict = {}

horseDict['lastRaceDay'] = bottomItems[0]
horseDict['lastRaceMonth'] = bottomItems[1]
horseDict['lastRaceYear'] = bottomItems[2]
horseDict['lastRaceTrack'] = bottomItems[3]
horseDict['lastRaceNum'] = topItems[0]
horseDict['lastRacePlace'] = topItems[1]
horseDict['program'] = bottomItems[4]
horseDict['horse'] = bottomItems[5]
horseDict['jockey'] = bottomItems[6]
horseDict['weight'] = bottomItems[7]
horseDict['m_e'] = bottomItems[8]
horseDict['placePP'] = bottomItems[9]
horseDict['placeSeg1'] = bottomItems[10]
horseDict['lengthsSeg1'] = topItems[2]
horseDict['placeSeg2'] = bottomItems[11]
horseDict['lengthsSeg2'] = topItems[3]
horseDict['placeSeg3'] = bottomItems[12]
horseDict['lengthsSeg3'] = topItems[4]
horseDict['placeSeg4'] = bottomItems[13]
horseDict['lengthsSeg4'] = topItems[5]
horseDict['placeSeg5'] = bottomItems[14]
horseDict['lengthsSeg5'] = topItems[6]
horseDict['placeSeg6'] = bottomItems[15]
horseDict['lengthsSeg6'] = topItems[7]
horseDict['odds'] = bottomItems[16]
horseDict['comments'] = bottomItems[17]

In [269]:
horseDict

{'lastRaceDay': '15',
 'lastRaceMonth': 'Dec',
 'lastRaceYear': '18',
 'lastRaceTrack': 'LRC',
 'lastRaceNum': '9',
 'lastRacePlace': '11',
 'program': '4',
 'horse': 'Spectacular Storm',
 'jockey': 'Vergara, Jr., Octavio',
 'weight': '123',
 'm_e': 'L b 3',
 'placePP': '5',
 'placeSeg1': '5',
 'lengthsSeg1': '21/2',
 'placeSeg2': '3',
 'lengthsSeg2': '7',
 'placeSeg3': '1',
 'lengthsSeg3': '6',
 'placeSeg4': '',
 'lengthsSeg4': '1',
 'placeSeg5': '',
 'lengthsSeg5': '',
 'placeSeg6': '',
 'lengthsSeg6': '',
 'odds': '0.80',
 'comments': 'svd ground, rail rally'}

In [225]:
pd.DataFrame(horseDict, index = [0])

,lastRaceDay,lastRaceMonth,lastRaceYear,lastRaceTrack,lastRaceNum,lastRacePlace,program,horseAndJockey,weight,m_e,...,placeSeg3,lengthsSeg3,placeSeg4,lengthsSeg4,placeSeg5,lengthsSeg5,placeFin,lengthsFin,odds,comments
0,15,Dec,18,LRC,9,11,4,"Spectacular Storm (Vergara, Jr., Octavio)",123,L b 3,...,1,6,,1,,,,,0.80,"svd ground, rail rally"


In [277]:
def parseHorseTopLine(line):
    print(line)
    fullSearch = re.search(r'^ ([0-9/A-Za-z]*) ?([0-9/A-Za-z]*) ?([0-9/A-Za-z]*) ?([0-9/A-Za-z]*) ?([0-9/A-Za-z]*) ?([0-9/A-Za-z]*) ?([0-9/A-Za-z]*) ?([0-9/A-Za-z]*)', line)

    out = []
    for i in range(1,9):
        out.append(fullSearch.group(i))

    return out

In [278]:
def parseHorseBottomLine(line):
    print(line)
    fullSearch = re.search(r'^ (\d?\d[A-Z][a-z]{2}\d\d [A-Z]{2,3}|---) (\d?\d) ([^0-9]+) (\d?\d?\d)[»½]* ([A-Za-z -]*\d?) ([-0-9]*) ?([-0-9]*) ?([-0-9]*) ?([-0-9]*) ?([-0-9]*) ?([-0-9]*) ?([-0-9]*) ([0-9]+\.\d\d)\*? (.*)$', line)

    out = []
    dateSearch = re.search(r'(\d?\d)([A-Z][a-z]{2})(\d\d) ([A-Z]{2,3})', fullSearch.group(1))
    if dateSearch is not None:
        out[:4] = [dateSearch.group(1), dateSearch.group(2), dateSearch.group(3), dateSearch.group(4)]
    else:
        out[:4] = [''] * 4
    
    out.append(fullSearch.group(2))

    horseAndJockey = fullSearch.group(3)
    hjSearch = re.search(r'(.*) \(([-A-Za-z,. ]+)\)', horseAndJockey)
    out.append(hjSearch.group(1))
    out.append(hjSearch.group(2))

    for i in range(4,15):
        out.append(fullSearch.group(i))

    return out

In [263]:
len(parseHorseBottomLine(line))

17

In [170]:
jack = re.search('(penis) (mcgee)', '   penis mcgee dickling')

<function Match.group>